<a href="https://colab.research.google.com/github/JGranda11/Proyecto-ai4eng20241/blob/main/10%20-%20preprocesado_P3_DS_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Preprocesamiento
En el siguiente cuaderno se quiere mostrar que realizaciones y consideraciones se tuvieron en cuenta para aplicar las operaciones de limpieza y preprocesado de datos al archivo de `test.csv.`

In [1]:
#Cargar el token de Kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jhonatangranda","key":"7dd2bc4d9b9fc9fcaeb5d991e5472b44"}'}

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [3]:
!kaggle competitions download udea-ai4eng-20241

 80% 17.0M/21.2M [00:00<00:00, 86.1MB/s]
100% 21.2M/21.2M [00:00<00:00, 85.5MB/s]


In [4]:
#Descomprimiendo los archivos
!unzip udea-ai4eng-20241

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import OrdinalEncoder

## Lectura inicial archivos
Queremos empezar a preprocesar el archivo de test, de la forma numero 3, que es la definitiva

In [6]:
test = pd.read_csv('test.csv')

In [7]:
test.shape

(296786, 16)

In [8]:
test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa


#Consideraciones
Del proceso de exploración de los datos, se infiere que el dataset en su composición tiene:

* Columnas repetidas:
__FAMI_TIENEINTERNET__ y __FAMI_TIENEINTERNET.1__

Adeás necesito categorizar varias columnas que representan un orden en su estructura
* ESTU_VALORMATRICULAUNIVERSIDAD
* ESTU_HORASSEMANATRABAJA
* FAMI_ESTRATOVIVIENDA
* FAMI_EDUCACIONPADRE
* FAMI_EDUCACIONMADRE

 Por lo que asignar numericamente un numero consecutivo, representa una categoria

In [9]:
#Se hace la asignación, para el cambio de nombre de la columna repetida
test = test.rename(columns={'FAMI_TIENEINTERNET.1': 'FAMI_TIENEINTERNET2'})

In [10]:
test.FAMI_TIENEINTERNET.value_counts()

FAMI_TIENEINTERNET
Si    254075
No     31172
Name: count, dtype: int64

In [11]:
test.FAMI_TIENEINTERNET2.value_counts()
#COLUMNA DUPLICADA

FAMI_TIENEINTERNET2
Si    254075
No     31172
Name: count, dtype: int64

In [12]:
validar = test['FAMI_TIENEINTERNET'].equals(test['FAMI_TIENEINTERNET2'])
print(validar)

True


In [13]:
test = test.drop('FAMI_TIENEINTERNET2', axis=1)

Evaluación general de todos los datos nulos

In [14]:
k = test.isna().sum()
k[k!=0]


ESTU_VALORMATRICULAUNIVERSIDAD     2723
ESTU_HORASSEMANATRABAJA           13379
FAMI_ESTRATOVIVIENDA              13795
FAMI_TIENEINTERNET                11539
FAMI_EDUCACIONPADRE                9993
FAMI_TIENELAVADORA                17259
FAMI_TIENEAUTOMOVIL               18918
ESTU_PAGOMATRICULAPROPIO           2807
FAMI_TIENECOMPUTADOR              16439
FAMI_EDUCACIONMADRE               10223
dtype: int64

## Manejo de la columna PERIODO
Esta columna presenta 9 distintas posibilidades de ocurrencia, se quiere corregir y agrupar de mejor manera por año.

In [15]:
test['PERIODO'] = test['PERIODO'].astype('category')
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2018.*', '2018', regex=True)
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2019.*', '2019', regex=True)
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2020.*', '2020', regex=True)
test['PERIODO'] = test['PERIODO'].astype(str).str.replace(r'2021.*', '2021', regex=True)

In [16]:
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificador = encode.fit_transform(test[['PERIODO']])
#generar codificacion

arreglo = codificador.toarray()
periodo = pd.DataFrame(arreglo,
                       columns = encode.categories_)

## Codificación de las columnas, manejo de OneHotEncoder

test = pd.concat([test, periodo], axis = 'columns')
test = test.drop('PERIODO', axis=1)

## Manejo de la columna ESTU_PRGM_ACADEMICO

In [17]:
#La columna ESTU_PRGM_ACADEMICO es la que representa un mayor numero de variedad en el rango de sus posibilidades
test["ESTU_PRGM_ACADEMICO"].value_counts()

ESTU_PRGM_ACADEMICO
DERECHO                                                                22984
ADMINISTRACION DE EMPRESAS                                             22298
CONTADURIA PUBLICA                                                     16861
PSICOLOGIA                                                             13818
INGENIERIA INDUSTRIAL                                                  12516
                                                                       ...  
DISEÑO Y GESTIÓN DEL PRODUCTO                                              1
LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN IDIOMAS EXTRANJEROS        1
ADMINISTRACION TECNOLOGICA                                                 1
LICENCIATURA EN EDUCACION ENFASIS EN HUMANIDADES                           1
AGROINDUSTRIA                                                              1
Name: count, Length: 919, dtype: int64

In [18]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00


In [19]:
from unidecode import unidecode
import re

test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].str.upper() # Todo mayúsculas
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: unidecode(x).strip()) # Eliminar tildes
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ',x).strip()) # Eliminar dobles espacios
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'^PROFESIONAL EN\s+', '', x).strip()) # "PROFESIONAL EN ..." redundante
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'INGENIER\?A', 'INGENIERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'INGENIIERIA', 'INGENIERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'EDUCACI\?N\b', 'EDUCACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'RECREACI\?N\b', 'RECREACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'F\?SICA\b', 'FISICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ESC\?NICAS\b', 'ESCENICAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'MATEM\?TICAS\b', 'MATEMATICAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOF\?A\b', 'FILOSOFIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISE\?O\b', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISE\?\?O\b', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\?N\b', 'COMUNICACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\?\?N\b', 'COMUNICACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GR\?\?FICA\b', 'GRAFICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'EL\?CTRICA\b', 'ELECTRICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'QU\?MICA\b', 'QUIMICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FARMAC\?UTICA\b', 'FARMACEUTICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACI\?N\b', 'ADMINISTRACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'P\?BLICA\b', 'PUBLICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'INGL\?S\b', 'INGLES', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'\?NFASIS\b', 'ENFASIS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GESTI\?N\b', 'GESTION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'M\?SICA\b', 'MUSICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ECONOM\?A\b', 'ECONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GASTRONOM\?A\b', 'GASTRONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'B\?SICA\b', 'BASICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GEOLOG\?A\b', 'GEOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'PEDAGOG\?A\b', 'PEDAGOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ART\?STICA\b', 'ARTISTICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'LOG\?STICA\b', 'LOGISTICA', regex=True)
# Reemplazar cualquier coincidencia de "{NOMBRE + AGG} ..." solo por "{NOMBRE} mas general"
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*INGENIERIA.*', 'INGENIERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACION.*', 'ADMINISTRACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISEÑO.*', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DISENO.*', 'DISEÑO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'LICENCIATURA.*', 'LICENCIATURA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CIENCIA.*', 'CIENCIAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACION.*', 'COMUNICACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GESTION.*', 'GESTION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOFIA.*', 'FILOSOFIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'QUIMICA.*', 'QUIMICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'NEGOCIOS.*', 'NEGOCIOS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*MERCAD.*', 'MERCADEO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'MATEMATICAS.*', 'MATEMATICAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*ARTE.*', 'ARTES', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ESTUDIOS.*', 'ESTUDIOS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'FINANZAS.*', 'FINANZAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CONSTRUCCION.*', 'CONSTRUCCION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'DIRECCION.*', 'DIRECCION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'RELACIONES.*', 'RELACIONES', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GEO.*', 'GEOGRAFIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'PROFESIONAL.*', 'PROFESIONAL', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'LENG.*', 'LENGUAS', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ADMIN.*', 'ADMINISTRACION', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*DEPORT.*', 'DEPORTE', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*BIOLOGIA.*', 'BIOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CINE.*', 'CINE', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'CONTADURIA.*', 'CONTADURIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'COMERCIO.*', 'COMERCIO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*LITERA.*', 'LITERATURA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*FILOL.*', 'FILOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'PUBLICI.*', 'PUBLICIDAD', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'MARK.*', 'MERCADEO', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'HISTORIA.*', 'HISTORIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*MUSICA.*', 'MUSICA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*VETE.*', 'VETERINARIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*ZOO.*', 'VETERINARIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*GASTR.*', 'GASTRONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*QUIRURGICA.*', 'MEDICINA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'ECONO.*', 'ECONOMIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'GEREN.*', 'GERENCIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'HOTELERIA.*', 'HOTELERIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*PSICOLO.*', 'PSICOLOGIA', regex=True)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].replace(r'.*AGRO.*', 'AGRONOMIA', regex=True)

Lo que se hizo fue reducir el rango de las columna 'ESTU_PRGM_ACADEMICO', para que se generan muchas columnas al aplicar Codificación, mayormente llenas de ceros

In [20]:
len(test['ESTU_PRGM_ACADEMICO'].unique())

109

In [21]:
carreras_agrupadas = ['INGENIERIA','ADMINISTRACION','DISEÑO','LICENCIATURA','CIENCIAS','COMUNICACION','GESTION','FILOSOFIA','QUIMICA','NEGOCIOS','MERCADEO',
                       'MATEMATICAS','ARTES','ESTUDIOS','FINANZAS','CONSTRUCCION','DIRECCION','RELACIONES','GEOGRAFIA','PROFESIONAL','LENGUAS', 'DEPORTE'
                       'BIOLOGIA','CINE','CONTADURIA','COMERCIO','LITERATURA','FILOLOGIA','PUBLICIDAD','HISTORIA','MUSICA','VETERINARIA','GATRONOMIA'
                       'MEDICINA', 'ECONOMIA','GERENCIA','HOTELERIA','PSICOLOGIA','AGRONOMIA']
len(carreras_agrupadas)

37

In [22]:
carreras_sobrantes = [elem for elem in test['ESTU_PRGM_ACADEMICO'] if elem not in carreras_agrupadas]

condicion = test['ESTU_PRGM_ACADEMICO'].isin(carreras_sobrantes)
test.loc[condicion, 'ESTU_PRGM_ACADEMICO'] = 'Otras'


In [23]:
academico=test['ESTU_PRGM_ACADEMICO'].unique()
len(academico)
#academico

36

In [24]:
##CODIFICAR PRGM ACADEMICO
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(test[['ESTU_PRGM_ACADEMICO']])
#generar codificacion

arregloPr = codificadorPr.toarray()
programa = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
test = pd.concat([test, programa], axis = 'columns')
test = test.drop('ESTU_PRGM_ACADEMICO', axis=1)

## Manejo de la columna ESTU_PRGM_DEPARTAMENTO
Con el manejo de esta columna lo que se quiere es agrupar por regiones dichos departamentos, es decir, pasar de tener 32 tipos de departamentos, a solo cinco regiones que los agrupen

In [25]:
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ANTIOQUIA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOYACA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOGOTÁ', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CALDAS', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CUNDINAMARCA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'HUILA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'NORTE SANTANDER', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'QUINDIO', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'RISARALDA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SANTANDER', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'TOLIMA', 'ANDINA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'AMAZONAS', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CAQUETA', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VAUPES', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'GUAVIARE', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'PUTUMAYO', 'AMAZONA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VALLE', 'PACIFICA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CHOCO', 'PACIFICA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CAUCA', 'PACIFICA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'NARIÑO', 'PACIFICA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ATLANTICO', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'BOLIVAR', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CESAR', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CORDOBA', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'LA GUAJIRA', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'MAGDALENA', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SUCRE', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'SAN ANDRES', 'CARIBE')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'ARAUCA', 'ORINOQUIA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'CASANARE', 'ORINOQUIA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'META', 'ORINOQUIA')
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].astype(str).str.replace(r'VICHADA', 'ORINOQUIA')

In [26]:
test['ESTU_PRGM_DEPARTAMENTO'].unique()

array(['CARIBE', 'ANDINA', 'PACIFICA', 'ORINOQUIA', 'AMAZONA'],
      dtype=object)

In [27]:
##CODIFICAR PRGM DEPARTAMENTO
#crear la instancia de OneHotEncoder
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorDepto = encode.fit_transform(test[['ESTU_PRGM_DEPARTAMENTO']])
#generar codificacion

arregloDepto = codificadorDepto.toarray()
departamento = pd.DataFrame(arregloDepto,
                       columns = encode.categories_)
#Suma de dataframes
test = pd.concat([test, departamento], axis = 'columns')
test = test.drop('ESTU_PRGM_DEPARTAMENTO', axis=1)

## Manejo de la columna ESTU_VALORMATRICULAUNIVERSIDAD
La columna presenta una posibilidad de 8 valores a la hroa de elegir cualquier registro y 6287 datos son nulos

In [28]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      86923
Entre 2.5 millones y menos de 4 millones    55193
Menos de 500 mil                            34589
Entre 500 mil y menos de 1 millón           33564
Entre 4 millones y menos de 5.5 millones    29855
Más de 7 millones                           29061
Entre 5.5 millones y menos de 7 millones    16558
No pagó matrícula                            8320
Name: count, dtype: int64

In [29]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('No pagó matrícula')

In [30]:
valores_matricula = ['No pagó matrícula','Menos de 500 mil','Entre 500 mil y menos de 1 millón',
                     'Entre 1 millón y menos de 2.5 millones',
                     'Entre 2.5 millones y menos de 4 millones',
                     'Entre 4 millones y menos de 5.5 millones',
                     'Entre 5.5 millones y menos de 7 millones','Más de 7 millones']
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(test[['ESTU_VALORMATRICULAUNIVERSIDAD']])
arregloPr = codificadorPr.toarray()
matricula = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
test = pd.concat([test, matricula], axis = 'columns')
test = test.drop('ESTU_VALORMATRICULAUNIVERSIDAD',axis=1)

In [31]:
test.head()

,ID,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,...,"(ORINOQUIA,)","(PACIFICA,)","(Entre 1 millón y menos de 2.5 millones,)","(Entre 2.5 millones y menos de 4 millones,)","(Entre 4 millones y menos de 5.5 millones,)","(Entre 5.5 millones y menos de 7 millones,)","(Entre 500 mil y menos de 1 millón,)","(Menos de 500 mil,)","(Más de 7 millones,)","(No pagó matrícula,)"
0,550236,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,98545,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,499179,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,782980,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,785185,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Manejo de la columna ESTU_HORASSEMANATRABAJA
La columna presenta 5 categorias, y presenta 30857 datos nulos, por lo que no debo realizar alguna reparacion, paar este caso asumimos que si el campo esta vacio, es por que el estudiante no tiene unas horas de trabajo

In [32]:
test['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', 'Entre 21 y 30 horas', '0',
       'Entre 11 y 20 horas', 'Más de 30 horas', nan], dtype=object)

In [33]:
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].fillna('0')

In [34]:
horas_trabajo = ['0','Menos de 10 horas','Entre 11 y 20 horas',
                'Entre 21 y 30 horas',
                'Más de 30 horas']
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(test[['ESTU_HORASSEMANATRABAJA']])
arregloPr = codificadorPr.toarray()
horas_trabajo = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
test = pd.concat([test, horas_trabajo], axis = 'columns')
test = test.drop('ESTU_HORASSEMANATRABAJA',axis=1)

In [35]:
test.head()

,ID,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,...,"(Entre 5.5 millones y menos de 7 millones,)","(Entre 500 mil y menos de 1 millón,)","(Menos de 500 mil,)","(Más de 7 millones,)","(No pagó matrícula,)","(0,)","(Entre 11 y 20 horas,)","(Entre 21 y 30 horas,)","(Menos de 10 horas,)","(Más de 30 horas,)"
0,550236,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Primaria completa,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,98545,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Técnica o tecnológica completa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,499179,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Secundaria (Bachillerato) completa,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,782980,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,Primaria incompleta,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,785185,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Secundaria (Bachillerato) completa,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Manejo de la columna FAMI_ESTRATOVIVIENDA
Dicha columna presenta 7 categorias, y ademas de eso un total de 32137 datos nulos, la estrategia que se plantea para repaar dichos datos nulos se baso en ver cual era la categoria con mayor frecuencia

In [36]:
test['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
Estrato 2      99288
Estrato 3      90719
Estrato 1      48092
Estrato 4      27823
Estrato 5      10194
Estrato 6       5451
Sin Estrato     1424
Name: count, dtype: int64

In [37]:
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')

In [38]:
estratos = ['Sin Estrato','Estrato 1','Estrato 2',
            'Estrato 3', 'Estrato 4', 'Estrato 5',
            'Estrato 6']
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(test[['FAMI_ESTRATOVIVIENDA']])
arregloPr = codificadorPr.toarray()
estrato = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
test = pd.concat([test, estrato], axis = 'columns')
test = test.drop('FAMI_ESTRATOVIVIENDA',axis=1)

## Manejo de la columna FAMI_TIENEINTERNET
Para este caso, tengo solo dos posibles clases, por lo que debo codificar de manera binaria, ademas tengo 26629 datos nuelos, y se decide clasificar dichos datos que no aparecen, como un 'NO'

In [39]:
test['FAMI_TIENEINTERNET'].value_counts()

FAMI_TIENEINTERNET
Si    254075
No     31172
Name: count, dtype: int64

In [40]:
test['FAMI_TIENEINTERNET'] = test['FAMI_TIENEINTERNET'].fillna("No")

In [41]:
test['FAMI_TIENEINTERNET'] = test['FAMI_TIENEINTERNET'].replace(r'Si', 1.)
test['FAMI_TIENEINTERNET'] = test['FAMI_TIENEINTERNET'].replace(r'No', 0.)

## Manejo de la columna FAMI_TIENELAVADORA
Esta columna consideramos que no aporta al modelo, y se suprime del dataframe


In [42]:
test['FAMI_TIENELAVADORA'].value_counts()

FAMI_TIENELAVADORA
Si    241565
No     37962
Name: count, dtype: int64

In [43]:
test = test.drop(['FAMI_TIENELAVADORA'], axis=1)

## Manejo de la columna FAMI_TIENEAUTOMOVIL
La columna presenta una distinción entre valores de tipo binario, por lo que todos los datos pueden tomar solo dos valores, y presenta 43623 registros nulos

In [44]:
test['FAMI_TIENEAUTOMOVIL'].value_counts()

FAMI_TIENEAUTOMOVIL
No    176625
Si    101243
Name: count, dtype: int64

In [45]:
test['FAMI_TIENEAUTOMOVIL'] = test['FAMI_TIENEAUTOMOVIL'].fillna("No")

In [46]:
test['FAMI_TIENEAUTOMOVIL'] = test['FAMI_TIENEAUTOMOVIL'].replace(r'Si', 1.)
test['FAMI_TIENEAUTOMOVIL'] = test['FAMI_TIENEAUTOMOVIL'].replace(r'No', 0.)

## Manejo de la columna FAMI_TIENECOMPUTADOR
Comportmaiento binario.
Presenta 38103 registros nulos

In [47]:
test['FAMI_TIENECOMPUTADOR'].value_counts()

FAMI_TIENECOMPUTADOR
Si    255996
No     24351
Name: count, dtype: int64

In [48]:
test['FAMI_TIENECOMPUTADOR'] = test['FAMI_TIENECOMPUTADOR'].fillna("No")

In [49]:
test['FAMI_TIENECOMPUTADOR'] = test['FAMI_TIENECOMPUTADOR'].replace(r'Si', 1.)
test['FAMI_TIENECOMPUTADOR'] = test['FAMI_TIENECOMPUTADOR'].replace(r'No', 0.)

## Manejo de la columna ESTU_PAGOMATRICULAPROPIO
Esta columna presenta un rango binario en su conjunto de posibles valores, y un total de 6498 datos nulos.
Por el momento la decisión para hcaer la reparación, se elige de asignar 'NO' a los datos nulos

In [50]:
test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

ESTU_PAGOMATRICULAPROPIO
No    163869
Si    130110
Name: count, dtype: int64

In [51]:
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].fillna("No")

In [52]:
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].replace(r'Si', 1.)
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].replace(r'No', 0.)

## Manejo de la columna ESTU_PRIVADO_LIBERTAD
Esta columna tiene un comportamiento binario, y no presenta datos nulos, se tiene la decisión de eliminar la columna del dataset

In [53]:
test['ESTU_PRIVADO_LIBERTAD'].value_counts()

ESTU_PRIVADO_LIBERTAD
N    296773
S        13
Name: count, dtype: int64

In [54]:
test = test.drop(['ESTU_PRIVADO_LIBERTAD'], axis=1)

## Manejo de la COLUMNA FAMI_EDUCACIONPADRE
Esta columna presento varias modificaciones, entre ellas la agrupación de las etiquetas 'Ninguno', 'No sabe' y 'No aplica' en una sola, junto con lso datos nulos, ademas de eso, se considera importante el hecho de jerarquizar dichas etiquetas

In [55]:
test['FAMI_EDUCACIONPADRE'].value_counts()

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      54838
Primaria incompleta                     53432
Educación profesional completa          35412
Secundaria (Bachillerato) incompleta    30641
Técnica o tecnológica completa          27045
Primaria completa                       24131
Postgrado                               19287
Educación profesional incompleta        11744
Técnica o tecnológica incompleta         9677
Ninguno                                  9565
No sabe                                  7066
No Aplica                                3955
Name: count, dtype: int64

In [56]:
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].replace(r'Ninguno', 'No Aplica')
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].replace(r'No sabe', 'No Aplica')
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].fillna('No Aplica')

In [57]:
nivelesPadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(test[['FAMI_EDUCACIONPADRE']])
#generar codificacion

arregloPr = codificadorPr.toarray()
edu_padre = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
test = pd.concat([test, edu_padre], axis = 'columns')
test = test.drop('FAMI_EDUCACIONPADRE',axis=1)

In [58]:
test.head()

,ID,FAMI_TIENEINTERNET,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,"(2018,)","(2019,)","(2020,)","(2021,)",...,"(Educación profesional completa,)","(Educación profesional incompleta,)","(No Aplica,)","(Postgrado,)","(Primaria completa,)","(Primaria incompleta,)","(Secundaria (Bachillerato) completa,)","(Secundaria (Bachillerato) incompleta,)","(Técnica o tecnológica completa,)","(Técnica o tecnológica incompleta,)"
0,550236,1.0,0.0,1.0,1.0,Primaria completa,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,98545,1.0,0.0,0.0,1.0,Técnica o tecnológica completa,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,499179,1.0,0.0,0.0,1.0,Secundaria (Bachillerato) completa,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,782980,0.0,0.0,0.0,0.0,Primaria incompleta,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,785185,1.0,0.0,0.0,1.0,Secundaria (Bachillerato) completa,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Manejo de la columna FAMI_EDUCACIONMADRE
Esta columna presento varias modificaciones, entre ellas la agrupación de las etiquetas 'Ninguno', 'No sabe' y 'No aplica' en una sola, ademas de eso, se considera importante el hecho de jerarquizar dichas etiquetas

In [59]:
test['FAMI_EDUCACIONMADRE'].value_counts()

FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      60649
Primaria incompleta                     42149
Técnica o tecnológica completa          38571
Educación profesional completa          36477
Secundaria (Bachillerato) incompleta    34566
Primaria completa                       24399
Postgrado                               20049
Técnica o tecnológica incompleta        11650
Educación profesional incompleta         9647
Ninguno                                  6335
No sabe                                  1286
No Aplica                                 785
Name: count, dtype: int64

In [60]:
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].replace(r'Ninguno', 'No Aplica')
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].replace(r'No sabe', 'No Aplica')
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].fillna('No Aplica')

In [61]:
nivelesMadre = ['No Aplica','Primaria incompleta','Primaria completa',
            'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
            'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa',
            'Postgrado']
encode = OneHotEncoder()

#Trabaja con dataframes, no series
codificadorPr = encode.fit_transform(test[['FAMI_EDUCACIONMADRE']])
#generar codificacion

arregloPr = codificadorPr.toarray()
edu_madre = pd.DataFrame(arregloPr,
                       columns = encode.categories_)
test = pd.concat([test, edu_madre], axis = 'columns')
test = test.drop('FAMI_EDUCACIONMADRE',axis=1)

In [62]:
test.head()

,ID,FAMI_TIENEINTERNET,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,"(2018,)","(2019,)","(2020,)","(2021,)","(ADMINISTRACION,)",...,"(Educación profesional completa,)","(Educación profesional incompleta,)","(No Aplica,)","(Postgrado,)","(Primaria completa,)","(Primaria incompleta,)","(Secundaria (Bachillerato) completa,)","(Secundaria (Bachillerato) incompleta,)","(Técnica o tecnológica completa,)","(Técnica o tecnológica incompleta,)"
0,550236,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,98545,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,499179,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,782980,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,785185,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [63]:
test.shape

(296786, 90)

In [64]:
print(len(test.columns))
test.columns

90


Index([                                         'ID',
                                'FAMI_TIENEINTERNET',
                               'FAMI_TIENEAUTOMOVIL',
                          'ESTU_PAGOMATRICULAPROPIO',
                              'FAMI_TIENECOMPUTADOR',
                                           ('2018',),
                                           ('2019',),
                                           ('2020',),
                                           ('2021',),
                                 ('ADMINISTRACION',),
                                      ('AGRONOMIA',),
                                          ('ARTES',),
                                       ('CIENCIAS',),
                                           ('CINE',),
                                       ('COMERCIO',),
                                   ('COMUNICACION',),
                                   ('CONSTRUCCION',),
                                     ('CONTADURIA',),
                            

In [65]:
archivo = test.to_csv("testP3.csv", index = False)

In [66]:
!head testP3.csv

ID,FAMI_TIENEINTERNET,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,"('2018',)","('2019',)","('2020',)","('2021',)","('ADMINISTRACION',)","('AGRONOMIA',)","('ARTES',)","('CIENCIAS',)","('CINE',)","('COMERCIO',)","('COMUNICACION',)","('CONSTRUCCION',)","('CONTADURIA',)","('DIRECCION',)","('DISEÑO',)","('ECONOMIA',)","('ESTUDIOS',)","('FILOLOGIA',)","('FILOSOFIA',)","('FINANZAS',)","('GEOGRAFIA',)","('GERENCIA',)","('GESTION',)","('HISTORIA',)","('HOTELERIA',)","('INGENIERIA',)","('LENGUAS',)","('LICENCIATURA',)","('LITERATURA',)","('MATEMATICAS',)","('MERCADEO',)","('MUSICA',)","('NEGOCIOS',)","('Otras',)","('PROFESIONAL',)","('PSICOLOGIA',)","('PUBLICIDAD',)","('QUIMICA',)","('RELACIONES',)","('VETERINARIA',)","('AMAZONA',)","('ANDINA',)","('CARIBE',)","('ORINOQUIA',)","('PACIFICA',)","('Entre 1 millón y menos de 2.5 millones',)","('Entre 2.5 millones y menos de 4 millones',)","('Entre 4 millones y menos de 5.5 millones',)","('Entre 5.5 millones y menos de 7 millone